In [1]:
!nvidia-smi

Thu May 16 03:59:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-rg1frv58
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-rg1frv58
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [12]:
!pip install nvcc4jupyter

In [13]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [14]:
%%cuda

#include <stdio.h>

__global__ void matrixMultiply(int *a, int *b, int *c, int width) {

 int row = blockIdx.y * blockDim.y + threadIdx.y;

 int col = blockIdx.x * blockDim.x + threadIdx.x;



if (row < width && col < width) {

 int sum = 0;

 for (int k = 0; k < width; ++k) {

 sum += a[row * width + k] * b[k * width + col];

 }

 c[row * width + col] = sum;

 }

}





void displayMatrix(int *matrix, int width) {

 for (int i = 0; i < width; ++i) {

 for (int j = 0; j < width; ++j) {

 printf("%d\t", matrix[i * width + j]);

 }

 printf("\n");

 }

}



int main() {

 const int width = 3;

 const int size = width * width * sizeof(int);



#  // Host matrices

 int h_mat1[width][width] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};

 int h_mat2[width][width] = {{9, 8, 7}, {6, 5, 4}, {3, 2, 1}};

 int h_result[width][width];



 // Device matrices

 int *d_mat1, *d_mat2, *d_result;

 cudaMalloc((void **)&d_mat1, size);

 cudaMalloc((void **)&d_mat2, size);
 cudaMalloc((void **)&d_result, size);



 // Copy matrices from host to device

 cudaMemcpy(d_mat1, h_mat1, size, cudaMemcpyHostToDevice);

 cudaMemcpy(d_mat2, h_mat2, size, cudaMemcpyHostToDevice);



 // Define grid and block dimensions

 dim3 threadsPerBlock(16, 16);

 dim3 blocksPerGrid((width + 15) / 16, (width + 15) / 16);



 // Launch kernel

 matrixMultiply<<<blocksPerGrid, threadsPerBlock>>>(d_mat1, d_mat2, d_result, width);



 // Copy result back to host

 cudaMemcpy(h_result, d_result, size, cudaMemcpyDeviceToHost);



 // Display the result

 printf("Result of matrix multiplication:\n");

 displayMatrix((int *)h_result, width);



 // Free device memory

 cudaFree(d_mat1);

 cudaFree(d_mat2);

 cudaFree(d_result);



 return 0;

}

Result of matrix multiplication:
0	0	0	
0	0	0	
0	0	0	

